# ECONOMIC LINKAGES: A Text-Based Approach (in development)
### Unique company peer-groups using topic modelling and clustering algorithms on text from quarterly earnings calls and company business descriptions
- Input data from company conference calls (fool.com) and long business descriptions (IEX)
- Technical hat-tip to [machinelearningplus.com](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/) for providing comprehensive instruction for python's gensim library

### Imports

In [104]:
# data manipulation
import pandas as pd
import numpy as np

# gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim import similarities
from gensim.parsing.porter import PorterStemmer

# aws 
from utils_s3 import get_etf_holdings, list_keys

# data viz
import pyLDAvis
import pyLDAvis.gensim

# pesky ipython warnings
import warnings
warnings.filterwarnings('ignore')

### Intiailize Raw Conference Call Data

#### Read Raw Input Data

In [64]:
# get list of quarterly calls (events) generated from download_transcripts.py
calls_raw = pd.read_csv('./extracts/foolcalls_extract_20200814.csv')

#### Filter Data
Only keep the _**presentation section**_ from calls from _**2019**_ for the _**largest 1000 companies**_ of the Russell 3000 (i.e. R1000)

In [65]:
# only calls from 2019
calls_2019 = calls_raw.loc[calls_raw['fiscal_period_year'] == 2019, ]

# remove statement_types that are either unknown or operator (i.e. keeping P, Q, A)
calls_2019_P = calls_2019.loc[calls_raw['statement_type'].isin(['P'])]

# only analyze 500 largest stocks from the R3000
russell_1000 = get_etf_holdings('IWV', '2020-07-31').sort_values('weight',ascending=False).head(1000)
calls_2019_P_R1000 = calls_2019_P.merge(russell_1000,on='ticker',how='inner')

#### Clean & Structure

In [68]:
# remove rows with missing text (nan)
calls_2019_P_R1000 = calls_2019_P_R1000.dropna(subset=['text'])

# join text by ticker (i.e. combine individual statements from the same call and combine calls from quarters/years)
calls = calls_2019_P_R1000.loc[:, ['ticker','text']].groupby(['ticker'])['text'].apply(lambda x: ''.join(x)).reset_index()

# assign one company name per ticker because they aren't consistent (e.g. Apple Inc vs Apple, Inc)
ticker_info = calls_2019_P_R1000[['ticker','company_name','weight','sector']].groupby('ticker').head(1)

# merge back to final data frame
calls = calls.merge(ticker_info,on='ticker',how='inner')
calls['ticker_name'] = calls['ticker'] + ': ' + calls['company_name']

## Pre-Processing
!!! todo: lemmatize and remove proper nouns. executives/analysts names are probably screwing this up.

In [69]:
# tokenize and remove punctuation
calls['text'] = calls['text'].apply(lambda x: simple_preprocess(x, min_len=2, max_len=15, deacc=True))

# Build the bigram model
bigram = models.Phrases(calls['text'], min_count=5, threshold=50) # higher threshold fewer phrases.
bigram_model = models.phrases.Phraser(bigram)
calls['text'] = calls['text'].apply(lambda x: bigram_model[x])

# remove stopwords
calls['text'] = calls['text'].apply(lambda x: [remove_stopwords(w) for w in x])

# stemming (porter)
p = PorterStemmer()
calls['text'] = calls['text'].apply(lambda x: p.stem_documents(x))

# create dictionary object
dictionary = corpora.Dictionary(calls['text'])

# filter extremes
dictionary.filter_extremes(no_below=2, no_above=0.20)

# bag-of-words transformation
corpus = [dictionary.doc2bow(text) for text in calls['text']]

# tfidf transformation
tfidf = models.TfidfModel(corpus)  # fit model
corpus_tfidf = tfidf[corpus]  # apply model


## Methodology 1: LSI Model Transformation

In [70]:
# lsi transformation
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # apply model
# for t in lsi_model.show_topics(): # print topics
#    print(t)

### Document Similarities (LSI)

In [71]:
# cosine similarity matrix with lsi
index_lsi = similarities.MatrixSimilarity(corpus_lsi)
sims_lsi = index_lsi[corpus_lsi]

##### Construct & Format Outputs for Analysis

In [92]:
# get top 10 peers for each ticker
# n = len(sims_lsi)
# for i, s in enumerate(sims_lsi):
#     print(calls['ticker_infoticker_name'].iloc[i])
#     print(calls['ticker_name'].iloc[s.argsort()[::-1][:n][1:10]])



# get top peers for each ticker
peers_lsi = pd.DataFrame()
for i, s in enumerate(sims_lsi):
    peers_idx = np.where(s > 0.50)[0].tolist()
    this_df = pd.DataFrame({'company': [calls['ticker_name'].iloc[i]]*len(peers_idx),
                            'weight': [calls['weight'].iloc[i]]*len(peers_idx),
                            'sector': [calls['sector'].iloc[i]]*len(peers_idx),
                            'peer': calls['ticker_name'].iloc[peers_idx],
                            'value': s[peers_idx]})
    peers_lsi = pd.concat([peers_lsi, this_df])

#### Explore Company-Level Peers (LSI)

In [105]:
peers_lsi = peers_lsi.dropna()
peers_lsi.loc[peers_lsi.company.str.contains('NKE'),].sort_values('value',ascending=False)[1:]

,company,weight,sector,peer,value
832,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,UAA: Under Armour (A Shares),0.689003
198,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,CPRI: Michael Kors Holdings Limited,0.663660
736,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,SHOO: Steven Madden Ltd,0.646873
421,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,IFF: International Flavors & Fragrances Inc,0.617213
204,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,CROX: Crocs,0.599658
189,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,COLM: Columbia Sportswear,0.593833
662,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,PM: PHILIP MORRIS INTERNATIONAL,0.566164
237,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,DECK: Deckers Outdoor Corp,0.557128
902,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,WWW: Wolverine World Wide,0.556291
109,"NKE: Nike, Inc.",0.0037,Consumer Discretionary,BIO: Bio-Rad Laboratories Inc,0.509508


#### Export to Cytoscape

In [82]:
peers_lsi.to_csv('output/peers_lsi.csv',sep='|',index=False)

![test](files/output/peers_lsi.png)

#### Preliminary Network Output

## Methodology 2: LDA Topic Modelling

In [102]:
# lda transformation
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
corpus_lda = lda[corpus] # apply lda model to corpus
lda.print_topics()

# data visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.001879  0.015575       1        1  15.204471
8      0.023241 -0.024777       2        1  12.220073
5      0.001808 -0.002699       3        1  10.743529
9      0.003950  0.006490       4        1  10.734125
6      0.008856  0.008581       5        1  10.729670
3     -0.020682 -0.017173       6        1   9.158596
2     -0.019064  0.008512       7        1   9.070771
1      0.008006  0.009454       8        1   8.673753
0     -0.013297 -0.012330       9        1   6.843531
7      0.009060  0.008368      10        1   6.621482, topic_info=               Term         Freq        Total Category  logprob  loglift
346         deposit  2370.000000  2370.000000  Default  30.0000  30.0000
1504           game  1279.000000  1279.000000  Default  29.0000  29.0000
1449  constant_curr  2771.000000  2771.000000  Default  28.0000  28.0000
5175         barrel   858.000000   858.000000  Default  27.0000  27.0000
1776            box   449.000000   449.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1504           game    73.110116  1279.181910  Topic10  -6.4387  -0.1472
2135        therapi    67.635507   894.832923  Topic10  -6.5165   0.1323
2145          trial    68.921789  1053.654609  Topic10  -6.4977  -0.0122
218       subscript    69.170806  1549.507307  Topic10  -6.4941  -0.3943
826          mexico    63.288549   921.814133  Topic10  -6.5829   0.0362

[742 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
5988      1  0.033767  accrual_loan
5988      2  0.135067  accrual_loan
5988      3  0.050650  accrual_loan
5988      4  0.033767  accrual_loan
5988      5  0.033767  accrual_loan
...     ...       ...           ...
7174      6  0.046062         yahoo
7174      7  0.046062         yahoo
7174      8  0.138186         yahoo
7174      9  0.046062         yahoo
7174     10  0.046062         yahoo

[4864 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 6, 10, 7, 4, 3, 2, 1, 8])

![placeholder](files/output/pyldavis.png)

### Document Similarities (LDA)

In [ ]:
# cosine similarity matrix with lda
index_lda = similarities.MatrixSimilarity(corpus_lda)
sims_lda = index_lda[corpus_lda]

# get top 10 peers for each ticker
n = len(sims_lda)
for i, s in enumerate(sims_lda):
    print(calls['ticker_name'].iloc[i])
    print(calls['ticker_name'].iloc[s.argsort()[::-1][:n][0:10]])

# get top peers for each ticker
peers_lda = pd.DataFrame()
for i, s in enumerate(sims_lda):
    peers_idx = np.where(s > 0.95)[0].tolist()
    this_df = pd.DataFrame({'company': [calls['ticker_name'].iloc[i]]*len(peers_idx),
                            'peer': calls['ticker_name'].iloc[peers_idx],
                            'value': s[peers_idx]})
    peers_lda = pd.concat([peers_lda, this_df])

#### Explore Company-Level Peers (LDA)

In [ ]:
peers_lda.loc[peers_lda.company.str.contains('AMZN'),].sort_values('value',ascending=False)[1:]